# Project: Mistral-7B Chatbot

## Section 1: Setup and Imports

In [1]:
!pip install transformers torch accelerate bitsandbytes sentencepiece

In [2]:
!pip install gradio

In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
from typing import Dict, List
import gradio as gr
from huggingface_hub import notebook_login
from PyPDF2 import PdfReader
import io
import time

In [6]:
# Log in to Hugging Face Hub
notebook_login()

## Section 2: Model Loading

In [7]:
def load_model():
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"

    # Load model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        load_in_4bit=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

model, tokenizer = load_model()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Section 3: Chat Processing

In [52]:
def format_prompt(message, chat_history):
    """Format the prompt for the model"""
    prompt = "<s>"
    for user_msg, assistant_msg in chat_history:
        prompt += f"[INST] {user_msg} [/INST] {assistant_msg}</s><s>"
    prompt += f"[INST] {message} [/INST]"
    return prompt

In [ ]:
def generate_response(prompt, max_length=1024):
    """Generate response from the model"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the latest response
    response = response.split("[/INST]")[-1].strip()
    return response

## Section 4: Chatbot Function

In [53]:
def chat_response(message, history):
    """Main chatbot function"""
    prompt = format_prompt(message, history)
    response = generate_response(prompt)

    # Clear GPU memory
    torch.cuda.empty_cache()

    return response

## Section 4: Gardio Interface

In [ ]:
def create_gradio_interface():
    with gr.Blocks(title="Mistral Chatbot") as demo:
        gr.Markdown("# Mistral-7B Chatbot")
        gr.Markdown("Chat with this AI assistant powered by Mistral-7B")

        chat_interface = gr.ChatInterface(
            fn=chat_response,
            title="Mistral Chatbot",
            description="A conversational AI powered by Mistral-7B",
            examples=[
                "What is machine learning?",
                "Can you explain quantum computing?",
                "Write a short story about a robot",
            ]
        )

    return demo

# Launch the interface
demo = create_gradio_interface()
demo.launch(share=True)